### Purpose of this notebook: scrape Taiwan's Post Office to get the street and road names via [Post Office](https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207)

### Playwright

In [10]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def get_one_city(city_name):
    data = []
    failures = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)  # set to True to run in background
        page = await browser.new_page()

        # Load the page
        await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")

        # Select the city
        await page.select_option("#city", city_name)
        await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")

        # Get district list
        district_options = await page.locator('#cityarea option').all()
        districts = [await opt.get_attribute('value') for opt in district_options if await opt.get_attribute('value')]

        for district in districts:
            try:
                # Refresh page for clean state
                await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")
                await page.select_option("#city", city_name)
                await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
                await page.select_option("#cityarea", district)

                # Force dropdown mode for roads
                await page.check('input#Street_kind0')
                await page.wait_for_timeout(1000)

                # Wait until road options appear
                await page.wait_for_function("""
                    () => {
                        const el = document.querySelector('#street option');
                        return el && el.value && el.value !== '';
                    }
                """, timeout=8000)

                # Get road names
                road_options = await page.locator('#street option').all()
                roads = [await opt.get_attribute('value') for opt in road_options if await opt.get_attribute('value')]

                for road in roads:
                    data.append({
                        "city": city_name,
                        "district": district,
                        "road": road
                    })

                print(f"✅ {city_name} → {district}: {len(roads)} roads")

            except Exception as e:
                print(f"❌ {city_name} → {district}: Failed ({e})")
                failures.append((city_name, district))

        await browser.close()

    # Save results
    df = pd.DataFrame(data).drop_duplicates()
    df.to_csv(f"{city_name}_roads.csv", index=False)
    pd.DataFrame(failures, columns=["city", "district"]).to_csv(f"{city_name}_failures.csv", index=False)

    print(f"\n🎉 Done with {city_name}! Saved to '{city_name}_roads.csv'")
    return df

# Run for 臺北市
await get_one_city("臺北市")


✅ 臺北市 → 中正區: 105 roads
✅ 臺北市 → 大同區: 56 roads
✅ 臺北市 → 中山區: 76 roads
✅ 臺北市 → 松山區: 47 roads
✅ 臺北市 → 大安區: 70 roads
✅ 臺北市 → 萬華區: 105 roads
✅ 臺北市 → 信義區: 47 roads
✅ 臺北市 → 士林區: 131 roads
✅ 臺北市 → 北投區: 159 roads
✅ 臺北市 → 內湖區: 56 roads
✅ 臺北市 → 南港區: 105 roads
✅ 臺北市 → 文山區: 80 roads

🎉 Done with 臺北市! Saved to '臺北市_roads.csv'


,city,district,road
0,臺北市,中正區,八德路一段
1,臺北市,中正區,三元街
2,臺北市,中正區,大埔街
3,臺北市,中正區,中山北路一段
4,臺北市,中正區,中山南路
...,...,...,...
1032,臺北市,文山區,興順街
1033,臺北市,文山區,興德路
1034,臺北市,文山區,羅斯福路四段
1035,臺北市,文山區,羅斯福路五段


In [16]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def get_districts(page, city_name):
    await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")
    await page.select_option("#city", city_name)
    await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
    options = await page.locator('#cityarea option').all()
    return [await opt.get_attribute('value') for opt in options if await opt.get_attribute('value')]

async def get_roads_for_district(p, city_name, district):
    data = []
    try:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")

        # Step 1: Select city
        await page.select_option("#city", city_name)
        await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
        await asyncio.sleep(1)

        # Step 2: Select district + manually trigger the onchange event
        await page.select_option("#cityarea", district)
        await page.dispatch_event("#cityarea", "change")
        await asyncio.sleep(1)

        # Step 3: Select road name mode (Street_kind0)
        await page.check("#Street_kind0")
        await asyncio.sleep(1)

        # Step 4: Wait for roads to be loaded
        await page.wait_for_function("""
            () => {
                const el = document.querySelector('#street option');
                return el && el.value && el.value !== '';
            }
        """, timeout=10000)

        # Step 5: Scrape road options
        options = await page.locator("#street option").all()
        roads = [await opt.get_attribute("value") for opt in options if await opt.get_attribute("value")]

        for road in roads:
            data.append({
                "city": city_name,
                "district": district,
                "road": road
            })

        print(f"✅ {city_name} → {district}: {len(roads)} roads")

        await browser.close()
        return data

    except Exception as e:
        print(f"❌ {city_name} → {district}: Failed ({e})")
        return []

async def get_one_city(city_name):
    all_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # Get list of districts
        districts = await get_districts(page, city_name)
        await browser.close()

        # Scrape each district
        for district in districts:
            roads = await get_roads_for_district(p, city_name, district)
            all_data.extend(roads)

    # Save to CSV
    df = pd.DataFrame(all_data).drop_duplicates()
    df.to_csv(f"{city_name}_roads.csv", index=False)
    print(f"\n🎉 Done with {city_name}! Total roads: {len(df)}")
    return df

# Run it
await get_one_city("桃園市")


✅ 桃園市 → 中壢區: 892 roads
✅ 桃園市 → 平鎮區: 303 roads
✅ 桃園市 → 龍潭區: 272 roads
✅ 桃園市 → 楊梅區: 234 roads
✅ 桃園市 → 新屋區: 266 roads
✅ 桃園市 → 觀音區: 376 roads
✅ 桃園市 → 桃園區: 549 roads
✅ 桃園市 → 龜山區: 363 roads
✅ 桃園市 → 八德區: 241 roads
✅ 桃園市 → 大溪區: 220 roads
✅ 桃園市 → 復興區: 78 roads
✅ 桃園市 → 大園區: 322 roads
✅ 桃園市 → 蘆竹區: 318 roads

🎉 Done with 桃園市! Total roads: 4434


,city,district,road
0,桃園市,中壢區,七和一街
1,桃園市,中壢區,七和路
2,桃園市,中壢區,九和一街
3,桃園市,中壢區,九和二街
4,桃園市,中壢區,九和三街
...,...,...,...
4429,桃園市,蘆竹區,蘆竹
4430,桃園市,蘆竹區,蘆竹街
4431,桃園市,蘆竹區,蘆宏路
4432,桃園市,蘆竹區,蘆興南路


In [21]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def get_districts(page, city_name):
    await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")
    await page.select_option("#city", city_name)
    await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
    options = await page.locator('#cityarea option').all()
    return [await opt.get_attribute('value') for opt in options if await opt.get_attribute('value')]

async def get_roads_for_district(p, city_name, district):
    data = []
    try:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")

        # Step 1: Select city
        await page.select_option("#city", city_name)
        await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
        await asyncio.sleep(1)

        # Step 2: Select district + manually trigger the onchange event
        await page.select_option("#cityarea", district)
        await page.dispatch_event("#cityarea", "change")
        await asyncio.sleep(1)

        # Step 3: Select road name mode (Street_kind0)
        await page.check("#Street_kind0")
        await asyncio.sleep(1)

        # Step 4: Wait for roads to be loaded
        await page.wait_for_function("""
            () => {
                const el = document.querySelector('#street option');
                return el && el.value && el.value !== '';
            }
        """, timeout=10000)

        # Step 5: Scrape road options
        options = await page.locator("#street option").all()
        roads = [await opt.get_attribute("value") for opt in options if await opt.get_attribute("value")]

        for road in roads:
            data.append({
                "city": city_name,
                "district": district,
                "road": road
            })

        print(f"✅ {city_name} → {district}: {len(roads)} roads")

        await browser.close()
        return data

    except Exception as e:
        print(f"❌ {city_name} → {district}: Failed ({e})")
        return []

async def get_one_city(city_name):
    all_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # Get list of districts
        districts = await get_districts(page, city_name)
        await browser.close()

        # Scrape each district
        for district in districts:
            roads = await get_roads_for_district(p, city_name, district)
            all_data.extend(roads)

    # Save to CSV
    df = pd.DataFrame(all_data).drop_duplicates()
    df.to_csv(f"{city_name}_roads.csv", index=False)
    print(f"\n🎉 Done with {city_name}! Total roads: {len(df)}")
    return df

# Run it
await get_one_city("新北市")


✅ 新北市 → 萬里區: 81 roads
✅ 新北市 → 金山區: 66 roads
✅ 新北市 → 板橋區: 221 roads
✅ 新北市 → 汐止區: 120 roads
✅ 新北市 → 深坑區: 32 roads
✅ 新北市 → 石碇區: 91 roads
✅ 新北市 → 瑞芳區: 107 roads
✅ 新北市 → 平溪區: 44 roads
✅ 新北市 → 雙溪區: 82 roads
✅ 新北市 → 貢寮區: 52 roads
✅ 新北市 → 新店區: 264 roads
✅ 新北市 → 坪林區: 54 roads
✅ 新北市 → 烏來區: 23 roads
✅ 新北市 → 永和區: 68 roads
✅ 新北市 → 中和區: 114 roads
✅ 新北市 → 土城區: 117 roads
✅ 新北市 → 三峽區: 94 roads
✅ 新北市 → 樹林區: 120 roads
✅ 新北市 → 鶯歌區: 161 roads
✅ 新北市 → 三重區: 207 roads
✅ 新北市 → 新莊區: 192 roads
✅ 新北市 → 泰山區: 82 roads
✅ 新北市 → 林口區: 131 roads
✅ 新北市 → 蘆洲區: 45 roads
✅ 新北市 → 五股區: 81 roads
✅ 新北市 → 八里區: 100 roads
✅ 新北市 → 淡水區: 175 roads
✅ 新北市 → 三芝區: 99 roads
✅ 新北市 → 石門區: 38 roads

🎉 Done with 新北市! Total roads: 3061


,city,district,road
0,新北市,萬里區,二坪
1,新北市,萬里區,八斗
2,新北市,萬里區,下社
3,新北市,萬里區,大中山
4,新北市,萬里區,大仁街
...,...,...,...
3056,新北市,石門區,嘉祿街
3057,新北市,石門區,福安街
3058,新北市,石門區,猪槽潭
3059,新北市,石門區,臨海別墅


In [31]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def get_districts(page, city_name):
    await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")
    await page.select_option("#city", city_name)
    await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
    options = await page.locator('#cityarea option').all()
    return [await opt.get_attribute('value') for opt in options if await opt.get_attribute('value')]

async def get_roads_for_district(p, city_name, district):
    data = []
    try:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")

        # Step 1: Select city
        await page.select_option("#city", city_name)
        await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
        await asyncio.sleep(1)

        # Step 2: Select district + manually trigger the onchange event
        await page.select_option("#cityarea", district)
        await page.dispatch_event("#cityarea", "change")
        await asyncio.sleep(1)

        # Step 3: Select road name mode (Street_kind0)
        await page.check("#Street_kind0")
        await asyncio.sleep(1)

        # Step 4: Wait for roads to be loaded
        await page.wait_for_function("""
            () => {
                const el = document.querySelector('#street option');
                return el && el.value && el.value !== '';
            }
        """, timeout=10000)

        # Step 5: Scrape road options
        options = await page.locator("#street option").all()
        roads = [await opt.get_attribute("value") for opt in options if await opt.get_attribute("value")]

        for road in roads:
            data.append({
                "city": city_name,
                "district": district,
                "road": road
            })

        print(f"✅ {city_name} → {district}: {len(roads)} roads")

        await browser.close()
        return data

    except Exception as e:
        print(f"❌ {city_name} → {district}: Failed ({e})")
        return []

async def get_one_city(city_name):
    all_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # Get list of districts
        districts = await get_districts(page, city_name)
        await browser.close()

        # Scrape each district
        for district in districts:
            roads = await get_roads_for_district(p, city_name, district)
            all_data.extend(roads)

    # Save to CSV
    df = pd.DataFrame(all_data).drop_duplicates()
    df.to_csv(f"{city_name}_roads.csv", index=False)
    print(f"\n🎉 Done with {city_name}! Total roads: {len(df)}")
    return df

# Run it
await get_one_city("基隆市")


✅ 基隆市 → 仁愛區: 43 roads
✅ 基隆市 → 信義區: 36 roads
✅ 基隆市 → 中正區: 39 roads
✅ 基隆市 → 中山區: 27 roads
✅ 基隆市 → 安樂區: 22 roads
✅ 基隆市 → 暖暖區: 16 roads
✅ 基隆市 → 七堵區: 57 roads

🎉 Done with 基隆市! Total roads: 240


,city,district,road
0,基隆市,仁愛區,仁一路
1,基隆市,仁愛區,仁二路
2,基隆市,仁愛區,仁三路
3,基隆市,仁愛區,仁四路
4,基隆市,仁愛區,仁五路
...,...,...,...
235,基隆市,七堵區,福五街
236,基隆市,七堵區,福六街
237,基隆市,七堵區,綠葉街
238,基隆市,七堵區,麗景一街


In [32]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def get_districts(page, city_name):
    await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")
    await page.select_option("#city", city_name)
    await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
    options = await page.locator('#cityarea option').all()
    return [await opt.get_attribute('value') for opt in options if await opt.get_attribute('value')]

async def get_roads_for_district(p, city_name, district):
    data = []
    try:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")

        # Step 1: Select city
        await page.select_option("#city", city_name)
        await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
        await asyncio.sleep(1)

        # Step 2: Select district + manually trigger the onchange event
        await page.select_option("#cityarea", district)
        await page.dispatch_event("#cityarea", "change")
        await asyncio.sleep(1)

        # Step 3: Select road name mode (Street_kind0)
        await page.check("#Street_kind0")
        await asyncio.sleep(1)

        # Step 4: Wait for roads to be loaded
        await page.wait_for_function("""
            () => {
                const el = document.querySelector('#street option');
                return el && el.value && el.value !== '';
            }
        """, timeout=10000)

        # Step 5: Scrape road options
        options = await page.locator("#street option").all()
        roads = [await opt.get_attribute("value") for opt in options if await opt.get_attribute("value")]

        for road in roads:
            data.append({
                "city": city_name,
                "district": district,
                "road": road
            })

        print(f"✅ {city_name} → {district}: {len(roads)} roads")

        await browser.close()
        return data

    except Exception as e:
        print(f"❌ {city_name} → {district}: Failed ({e})")
        return []

async def get_one_city(city_name):
    all_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # Get list of districts
        districts = await get_districts(page, city_name)
        await browser.close()

        # Scrape each district
        for district in districts:
            roads = await get_roads_for_district(p, city_name, district)
            all_data.extend(roads)

    # Save to CSV
    df = pd.DataFrame(all_data).drop_duplicates()
    df.to_csv(f"{city_name}_roads.csv", index=False)
    print(f"\n🎉 Done with {city_name}! Total roads: {len(df)}")
    return df

# Run it
await get_one_city("新竹市")


✅ 新竹市 → 東區: 251 roads
✅ 新竹市 → 北區: 156 roads
✅ 新竹市 → 香山區: 113 roads

🎉 Done with 新竹市! Total roads: 520


,city,district,road
0,新竹市,東區,八德路
1,新竹市,東區,力行一路
2,新竹市,東區,力行二路
3,新竹市,東區,力行三路
4,新竹市,東區,力行四路
...,...,...,...
515,新竹市,香山區,遊樂街
516,新竹市,香山區,福樹街
517,新竹市,香山區,墩豐路
518,新竹市,香山區,樹下街


In [33]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def get_districts(page, city_name):
    await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")
    await page.select_option("#city", city_name)
    await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
    options = await page.locator('#cityarea option').all()
    return [await opt.get_attribute('value') for opt in options if await opt.get_attribute('value')]

async def get_roads_for_district(p, city_name, district):
    data = []
    try:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")

        # Step 1: Select city
        await page.select_option("#city", city_name)
        await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
        await asyncio.sleep(1)

        # Step 2: Select district + manually trigger the onchange event
        await page.select_option("#cityarea", district)
        await page.dispatch_event("#cityarea", "change")
        await asyncio.sleep(1)

        # Step 3: Select road name mode (Street_kind0)
        await page.check("#Street_kind0")
        await asyncio.sleep(1)

        # Step 4: Wait for roads to be loaded
        await page.wait_for_function("""
            () => {
                const el = document.querySelector('#street option');
                return el && el.value && el.value !== '';
            }
        """, timeout=10000)

        # Step 5: Scrape road options
        options = await page.locator("#street option").all()
        roads = [await opt.get_attribute("value") for opt in options if await opt.get_attribute("value")]

        for road in roads:
            data.append({
                "city": city_name,
                "district": district,
                "road": road
            })

        print(f"✅ {city_name} → {district}: {len(roads)} roads")

        await browser.close()
        return data

    except Exception as e:
        print(f"❌ {city_name} → {district}: Failed ({e})")
        return []

async def get_one_city(city_name):
    all_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # Get list of districts
        districts = await get_districts(page, city_name)
        await browser.close()

        # Scrape each district
        for district in districts:
            roads = await get_roads_for_district(p, city_name, district)
            all_data.extend(roads)

    # Save to CSV
    df = pd.DataFrame(all_data).drop_duplicates()
    df.to_csv(f"{city_name}_roads.csv", index=False)
    print(f"\n🎉 Done with {city_name}! Total roads: {len(df)}")
    return df

# Run it
await get_one_city("新竹縣")


✅ 新竹縣 → 竹北市: 368 roads
✅ 新竹縣 → 湖口鄉: 256 roads
✅ 新竹縣 → 新豐鄉: 91 roads
✅ 新竹縣 → 新埔鎮: 106 roads
✅ 新竹縣 → 關西鎮: 96 roads
✅ 新竹縣 → 芎林鄉: 67 roads
✅ 新竹縣 → 寶山鄉: 147 roads
✅ 新竹縣 → 竹東鎮: 146 roads
✅ 新竹縣 → 五峰鄉: 14 roads
✅ 新竹縣 → 橫山鄉: 48 roads
✅ 新竹縣 → 尖石鄉: 41 roads
✅ 新竹縣 → 北埔鄉: 43 roads
✅ 新竹縣 → 峨眉鄉: 35 roads

🎉 Done with 新竹縣! Total roads: 1458


,city,district,road
0,新竹縣,竹北市,十興一街
1,新竹縣,竹北市,十興二街
2,新竹縣,竹北市,十興三街
3,新竹縣,竹北市,十興五街
4,新竹縣,竹北市,十興路
...,...,...,...
1453,新竹縣,峨眉鄉,富興
1454,新竹縣,峨眉鄉,富興頭
1455,新竹縣,峨眉鄉,湖農新邨
1456,新竹縣,峨眉鄉,獅山街


In [34]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def get_districts(page, city_name):
    await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")
    await page.select_option("#city", city_name)
    await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
    options = await page.locator('#cityarea option').all()
    return [await opt.get_attribute('value') for opt in options if await opt.get_attribute('value')]

async def get_roads_for_district(p, city_name, district):
    data = []
    try:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")

        # Step 1: Select city
        await page.select_option("#city", city_name)
        await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
        await asyncio.sleep(1)

        # Step 2: Select district + manually trigger the onchange event
        await page.select_option("#cityarea", district)
        await page.dispatch_event("#cityarea", "change")
        await asyncio.sleep(1)

        # Step 3: Select road name mode (Street_kind0)
        await page.check("#Street_kind0")
        await asyncio.sleep(1)

        # Step 4: Wait for roads to be loaded
        await page.wait_for_function("""
            () => {
                const el = document.querySelector('#street option');
                return el && el.value && el.value !== '';
            }
        """, timeout=10000)

        # Step 5: Scrape road options
        options = await page.locator("#street option").all()
        roads = [await opt.get_attribute("value") for opt in options if await opt.get_attribute("value")]

        for road in roads:
            data.append({
                "city": city_name,
                "district": district,
                "road": road
            })

        print(f"✅ {city_name} → {district}: {len(roads)} roads")

        await browser.close()
        return data

    except Exception as e:
        print(f"❌ {city_name} → {district}: Failed ({e})")
        return []

async def get_one_city(city_name):
    all_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # Get list of districts
        districts = await get_districts(page, city_name)
        await browser.close()

        # Scrape each district
        for district in districts:
            roads = await get_roads_for_district(p, city_name, district)
            all_data.extend(roads)

    # Save to CSV
    df = pd.DataFrame(all_data).drop_duplicates()
    df.to_csv(f"{city_name}_roads.csv", index=False)
    print(f"\n🎉 Done with {city_name}! Total roads: {len(df)}")
    return df

# Run it
await get_one_city("苗栗縣")


✅ 苗栗縣 → 竹南鎮: 286 roads
✅ 苗栗縣 → 頭份市: 293 roads
✅ 苗栗縣 → 三灣鄉: 69 roads
✅ 苗栗縣 → 南庄鄉: 50 roads


Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed


✅ 苗栗縣 → 獅潭鄉: 46 roads
✅ 苗栗縣 → 後龍鎮: 165 roads
✅ 苗栗縣 → 通霄鎮: 66 roads
✅ 苗栗縣 → 苑裡鎮: 100 roads
✅ 苗栗縣 → 苗栗市: 300 roads
✅ 苗栗縣 → 造橋鄉: 94 roads
✅ 苗栗縣 → 頭屋鄉: 27 roads
✅ 苗栗縣 → 公館鄉: 55 roads
✅ 苗栗縣 → 大湖鄉: 88 roads
✅ 苗栗縣 → 泰安鄉: 24 roads
✅ 苗栗縣 → 銅鑼鄉: 72 roads
✅ 苗栗縣 → 三義鄉: 47 roads
✅ 苗栗縣 → 西湖鄉: 53 roads
✅ 苗栗縣 → 卓蘭鎮: 57 roads

🎉 Done with 苗栗縣! Total roads: 1892


,city,district,road
0,苗栗縣,竹南鎮,七崙仔
1,苗栗縣,竹南鎮,八德一路
2,苗栗縣,竹南鎮,八德街
3,苗栗縣,竹南鎮,三平路
4,苗栗縣,竹南鎮,三民街
...,...,...,...
1887,苗栗縣,卓蘭鎮,經國路
1888,苗栗縣,卓蘭鎮,電廠
1889,苗栗縣,卓蘭鎮,興南街
1890,苗栗縣,卓蘭鎮,豐田


In [36]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def get_districts(page, city_name):
    await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")
    await page.select_option("#city", city_name)
    await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
    options = await page.locator('#cityarea option').all()
    return [await opt.get_attribute('value') for opt in options if await opt.get_attribute('value')]

async def get_roads_for_district(p, city_name, district):
    data = []
    try:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")

        # Step 1: Select city
        await page.select_option("#city", city_name)
        await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
        await asyncio.sleep(1)

        # Step 2: Select district + manually trigger the onchange event
        await page.select_option("#cityarea", district)
        await page.dispatch_event("#cityarea", "change")
        await asyncio.sleep(1)

        # Step 3: Select road name mode (Street_kind0)
        await page.check("#Street_kind0")
        await asyncio.sleep(1)

        # Step 4: Wait for roads to be loaded
        await page.wait_for_function("""
            () => {
                const el = document.querySelector('#street option');
                return el && el.value && el.value !== '';
            }
        """, timeout=10000)

        # Step 5: Scrape road options
        options = await page.locator("#street option").all()
        roads = [await opt.get_attribute("value") for opt in options if await opt.get_attribute("value")]

        for road in roads:
            data.append({
                "city": city_name,
                "district": district,
                "road": road
            })

        print(f"✅ {city_name} → {district}: {len(roads)} roads")

        await browser.close()
        return data

    except Exception as e:
        print(f"❌ {city_name} → {district}: Failed ({e})")
        return []

async def get_one_city(city_name):
    all_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # Get list of districts
        districts = await get_districts(page, city_name)
        await browser.close()

        # Scrape each district
        for district in districts:
            roads = await get_roads_for_district(p, city_name, district)
            all_data.extend(roads)

    # Save to CSV
    df = pd.DataFrame(all_data).drop_duplicates()
    df.to_csv(f"{city_name}_roads.csv", index=False)
    print(f"\n🎉 Done with {city_name}! Total roads: {len(df)}")
    return df

# Run it
await get_one_city("臺中市")


✅ 臺中市 → 中區: 32 roads
✅ 臺中市 → 東區: 162 roads
✅ 臺中市 → 南區: 149 roads
✅ 臺中市 → 西區: 185 roads
✅ 臺中市 → 北區: 225 roads
✅ 臺中市 → 北屯區: 490 roads
✅ 臺中市 → 西屯區: 492 roads
✅ 臺中市 → 南屯區: 304 roads
✅ 臺中市 → 太平區: 344 roads
✅ 臺中市 → 大里區: 341 roads
✅ 臺中市 → 霧峰區: 137 roads
✅ 臺中市 → 烏日區: 158 roads
✅ 臺中市 → 豐原區: 214 roads
✅ 臺中市 → 后里區: 132 roads
✅ 臺中市 → 石岡區: 45 roads
✅ 臺中市 → 東勢區: 122 roads
✅ 臺中市 → 和平區: 74 roads
✅ 臺中市 → 新社區: 64 roads
✅ 臺中市 → 潭子區: 149 roads
✅ 臺中市 → 大雅區: 163 roads
✅ 臺中市 → 神岡區: 109 roads
✅ 臺中市 → 大肚區: 68 roads
✅ 臺中市 → 沙鹿區: 274 roads
✅ 臺中市 → 龍井區: 126 roads
✅ 臺中市 → 梧棲區: 176 roads
✅ 臺中市 → 清水區: 227 roads
✅ 臺中市 → 大甲區: 134 roads
✅ 臺中市 → 外埔區: 42 roads
✅ 臺中市 → 大安區: 54 roads

🎉 Done with 臺中市! Total roads: 5192


,city,district,road
0,臺中市,中區,三民路二段
1,臺中市,中區,三民路二段第二市場
2,臺中市,中區,三民路三段
3,臺中市,中區,大誠街
4,臺中市,中區,中山路
...,...,...,...
5187,臺中市,大安區,福東一路
5188,臺中市,大安區,福東二路
5189,臺中市,大安區,福東路
5190,臺中市,大安區,興安路


In [37]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def get_districts(page, city_name):
    await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")
    await page.select_option("#city", city_name)
    await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
    options = await page.locator('#cityarea option').all()
    return [await opt.get_attribute('value') for opt in options if await opt.get_attribute('value')]

async def get_roads_for_district(p, city_name, district):
    data = []
    try:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")

        # Step 1: Select city
        await page.select_option("#city", city_name)
        await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
        await asyncio.sleep(1)

        # Step 2: Select district + manually trigger the onchange event
        await page.select_option("#cityarea", district)
        await page.dispatch_event("#cityarea", "change")
        await asyncio.sleep(1)

        # Step 3: Select road name mode (Street_kind0)
        await page.check("#Street_kind0")
        await asyncio.sleep(1)

        # Step 4: Wait for roads to be loaded
        await page.wait_for_function("""
            () => {
                const el = document.querySelector('#street option');
                return el && el.value && el.value !== '';
            }
        """, timeout=10000)

        # Step 5: Scrape road options
        options = await page.locator("#street option").all()
        roads = [await opt.get_attribute("value") for opt in options if await opt.get_attribute("value")]

        for road in roads:
            data.append({
                "city": city_name,
                "district": district,
                "road": road
            })

        print(f"✅ {city_name} → {district}: {len(roads)} roads")

        await browser.close()
        return data

    except Exception as e:
        print(f"❌ {city_name} → {district}: Failed ({e})")
        return []

async def get_one_city(city_name):
    all_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # Get list of districts
        districts = await get_districts(page, city_name)
        await browser.close()

        # Scrape each district
        for district in districts:
            roads = await get_roads_for_district(p, city_name, district)
            all_data.extend(roads)

    # Save to CSV
    df = pd.DataFrame(all_data).drop_duplicates()
    df.to_csv(f"{city_name}_roads.csv", index=False)
    print(f"\n🎉 Done with {city_name}! Total roads: {len(df)}")
    return df

# Run it
await get_one_city("彰化縣")


✅ 彰化縣 → 彰化市: 265 roads
✅ 彰化縣 → 芬園鄉: 63 roads
✅ 彰化縣 → 花壇鄉: 98 roads
✅ 彰化縣 → 秀水鄉: 106 roads
✅ 彰化縣 → 鹿港鎮: 285 roads
✅ 彰化縣 → 福興鄉: 74 roads
✅ 彰化縣 → 線西鄉: 70 roads
✅ 彰化縣 → 和美鎮: 259 roads
✅ 彰化縣 → 伸港鄉: 148 roads
✅ 彰化縣 → 員林市: 259 roads
✅ 彰化縣 → 社頭鄉: 119 roads
✅ 彰化縣 → 永靖鄉: 123 roads
✅ 彰化縣 → 埔心鄉: 135 roads
✅ 彰化縣 → 溪湖鎮: 275 roads
✅ 彰化縣 → 大村鄉: 77 roads
✅ 彰化縣 → 埔鹽鄉: 53 roads
✅ 彰化縣 → 田中鎮: 129 roads
✅ 彰化縣 → 北斗鎮: 98 roads
✅ 彰化縣 → 田尾鄉: 91 roads
✅ 彰化縣 → 埤頭鄉: 104 roads
✅ 彰化縣 → 溪州鄉: 223 roads
✅ 彰化縣 → 竹塘鄉: 63 roads
✅ 彰化縣 → 二林鎮: 266 roads
✅ 彰化縣 → 大城鄉: 99 roads
✅ 彰化縣 → 芳苑鄉: 239 roads
✅ 彰化縣 → 二水鄉: 77 roads

🎉 Done with 彰化縣! Total roads: 3798


,city,district,road
0,彰化縣,彰化市,一心東街
1,彰化縣,彰化市,一心南街
2,彰化縣,彰化市,一心新村
3,彰化縣,彰化市,一德南路
4,彰化縣,彰化市,力行路
...,...,...,...
3793,彰化縣,二水鄉,福民巷
3794,彰化縣,二水鄉,鼻倡路
3795,彰化縣,二水鄉,庄尾一巷
3796,彰化縣,二水鄉,庄尾二巷


In [39]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def get_districts(page, city_name):
    await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")
    await page.select_option("#city", city_name)
    await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
    options = await page.locator('#cityarea option').all()
    return [await opt.get_attribute('value') for opt in options if await opt.get_attribute('value')]

async def get_roads_for_district(p, city_name, district):
    data = []
    try:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")

        # Step 1: Select city
        await page.select_option("#city", city_name)
        await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
        await asyncio.sleep(1)

        # Step 2: Select district + manually trigger the onchange event
        await page.select_option("#cityarea", district)
        await page.dispatch_event("#cityarea", "change")
        await asyncio.sleep(1)

        # Step 3: Select road name mode (Street_kind0)
        await page.check("#Street_kind0")
        await asyncio.sleep(1)

        # Step 4: Wait for roads to be loaded
        await page.wait_for_function("""
            () => {
                const el = document.querySelector('#street option');
                return el && el.value && el.value !== '';
            }
        """, timeout=10000)

        # Step 5: Scrape road options
        options = await page.locator("#street option").all()
        roads = [await opt.get_attribute("value") for opt in options if await opt.get_attribute("value")]

        for road in roads:
            data.append({
                "city": city_name,
                "district": district,
                "road": road
            })

        print(f"✅ {city_name} → {district}: {len(roads)} roads")

        await browser.close()
        return data

    except Exception as e:
        print(f"❌ {city_name} → {district}: Failed ({e})")
        return []

async def get_one_city(city_name):
    all_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # Get list of districts
        districts = await get_districts(page, city_name)
        await browser.close()

        # Scrape each district
        for district in districts:
            roads = await get_roads_for_district(p, city_name, district)
            all_data.extend(roads)

    # Save to CSV
    df = pd.DataFrame(all_data).drop_duplicates()
    df.to_csv(f"{city_name}_roads.csv", index=False)
    print(f"\n🎉 Done with {city_name}! Total roads: {len(df)}")
    return df

# Run it
await get_one_city("南投縣")


✅ 南投縣 → 南投市: 252 roads
✅ 南投縣 → 中寮鄉: 29 roads
✅ 南投縣 → 草屯鎮: 223 roads
✅ 南投縣 → 國姓鄉: 43 roads
✅ 南投縣 → 埔里鎮: 361 roads
✅ 南投縣 → 仁愛鄉: 61 roads
✅ 南投縣 → 名間鄉: 80 roads
✅ 南投縣 → 集集鎮: 87 roads
✅ 南投縣 → 水里鄉: 132 roads
✅ 南投縣 → 魚池鄉: 52 roads
✅ 南投縣 → 信義鄉: 55 roads
✅ 南投縣 → 竹山鎮: 205 roads
✅ 南投縣 → 鹿谷鄉: 62 roads

🎉 Done with 南投縣! Total roads: 1642


,city,district,road
0,南投縣,南投市,八卦路
1,南投縣,南投市,八德路
2,南投縣,南投市,力行一街
3,南投縣,南投市,力行二街
4,南投縣,南投市,力行三街
...,...,...,...
1637,南投縣,鹿谷鄉,興產路觀湖巷
1638,南投縣,鹿谷鄉,興農巷
1639,南投縣,鹿谷鄉,麒麟路
1640,南投縣,鹿谷鄉,籐湖巷


In [1]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def get_districts(page, city_name):
    await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")
    await page.select_option("#city", city_name)
    await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
    options = await page.locator('#cityarea option').all()
    return [await opt.get_attribute('value') for opt in options if await opt.get_attribute('value')]

async def get_roads_for_district(p, city_name, district):
    data = []
    try:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")

        # Step 1: Select city
        await page.select_option("#city", city_name)
        await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
        await asyncio.sleep(1)

        # Step 2: Select district + manually trigger the onchange event
        await page.select_option("#cityarea", district)
        await page.dispatch_event("#cityarea", "change")
        await asyncio.sleep(1)

        # Step 3: Select road name mode (Street_kind0)
        await page.check("#Street_kind0")
        await asyncio.sleep(1)

        # Step 4: Wait for roads to be loaded
        await page.wait_for_function("""
            () => {
                const el = document.querySelector('#street option');
                return el && el.value && el.value !== '';
            }
        """, timeout=10000)

        # Step 5: Scrape road options
        options = await page.locator("#street option").all()
        roads = [await opt.get_attribute("value") for opt in options if await opt.get_attribute("value")]

        for road in roads:
            data.append({
                "city": city_name,
                "district": district,
                "road": road
            })

        print(f"✅ {city_name} → {district}: {len(roads)} roads")

        await browser.close()
        return data

    except Exception as e:
        print(f"❌ {city_name} → {district}: Failed ({e})")
        return []

async def get_one_city(city_name):
    all_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # Get list of districts
        districts = await get_districts(page, city_name)
        await browser.close()

        # Scrape each district
        for district in districts:
            roads = await get_roads_for_district(p, city_name, district)
            all_data.extend(roads)

    # Save to CSV
    df = pd.DataFrame(all_data).drop_duplicates()
    df.to_csv(f"{city_name}_roads.csv", index=False)
    print(f"\n🎉 Done with {city_name}! Total roads: {len(df)}")
    return df

# Run it
await get_one_city("雲林縣")


✅ 雲林縣 → 斗南鎮: 194 roads
✅ 雲林縣 → 大埤鄉: 67 roads
✅ 雲林縣 → 虎尾鎮: 224 roads
✅ 雲林縣 → 土庫鎮: 78 roads
✅ 雲林縣 → 褒忠鄉: 32 roads
✅ 雲林縣 → 東勢鄉: 69 roads
✅ 雲林縣 → 臺西鄉: 46 roads
✅ 雲林縣 → 崙背鄉: 112 roads
✅ 雲林縣 → 麥寮鄉: 93 roads
✅ 雲林縣 → 斗六市: 386 roads
✅ 雲林縣 → 林內鄉: 58 roads
✅ 雲林縣 → 古坑鄉: 75 roads
✅ 雲林縣 → 莿桐鄉: 46 roads
✅ 雲林縣 → 西螺鎮: 111 roads
✅ 雲林縣 → 二崙鄉: 61 roads
✅ 雲林縣 → 北港鎮: 153 roads
✅ 雲林縣 → 水林鄉: 65 roads
✅ 雲林縣 → 口湖鄉: 83 roads
✅ 雲林縣 → 四湖鄉: 72 roads
✅ 雲林縣 → 元長鄉: 84 roads

🎉 Done with 雲林縣! Total roads: 2109


,city,district,road
0,雲林縣,斗南鎮,七賢街
1,雲林縣,斗南鎮,二重溝
2,雲林縣,斗南鎮,八德街
3,雲林縣,斗南鎮,三民路
4,雲林縣,斗南鎮,三和街
...,...,...,...
2104,雲林縣,元長鄉,興工街
2105,雲林縣,元長鄉,興安路
2106,雲林縣,元長鄉,龍岩
2107,雲林縣,元長鄉,贊庄


In [2]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def get_districts(page, city_name):
    await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")
    await page.select_option("#city", city_name)
    await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
    options = await page.locator('#cityarea option').all()
    return [await opt.get_attribute('value') for opt in options if await opt.get_attribute('value')]

async def get_roads_for_district(p, city_name, district):
    data = []
    try:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")

        # Step 1: Select city
        await page.select_option("#city", city_name)
        await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
        await asyncio.sleep(1)

        # Step 2: Select district + manually trigger the onchange event
        await page.select_option("#cityarea", district)
        await page.dispatch_event("#cityarea", "change")
        await asyncio.sleep(1)

        # Step 3: Select road name mode (Street_kind0)
        await page.check("#Street_kind0")
        await asyncio.sleep(1)

        # Step 4: Wait for roads to be loaded
        await page.wait_for_function("""
            () => {
                const el = document.querySelector('#street option');
                return el && el.value && el.value !== '';
            }
        """, timeout=10000)

        # Step 5: Scrape road options
        options = await page.locator("#street option").all()
        roads = [await opt.get_attribute("value") for opt in options if await opt.get_attribute("value")]

        for road in roads:
            data.append({
                "city": city_name,
                "district": district,
                "road": road
            })

        print(f"✅ {city_name} → {district}: {len(roads)} roads")

        await browser.close()
        return data

    except Exception as e:
        print(f"❌ {city_name} → {district}: Failed ({e})")
        return []

async def get_one_city(city_name):
    all_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # Get list of districts
        districts = await get_districts(page, city_name)
        await browser.close()

        # Scrape each district
        for district in districts:
            roads = await get_roads_for_district(p, city_name, district)
            all_data.extend(roads)

    # Save to CSV
    df = pd.DataFrame(all_data).drop_duplicates()
    df.to_csv(f"{city_name}_roads.csv", index=False)
    print(f"\n🎉 Done with {city_name}! Total roads: {len(df)}")
    return df

# Run it
await get_one_city("嘉義市")


✅ 嘉義市 → 東區: 169 roads
✅ 嘉義市 → 西區: 364 roads

🎉 Done with 嘉義市! Total roads: 533


,city,district,road
0,嘉義市,東區,大同新村
1,嘉義市,東區,大埤脚
2,嘉義市,東區,大雅路一段
3,嘉義市,東區,大雅路二段
4,嘉義市,東區,大業街
...,...,...,...
528,嘉義市,西區,蘭州一街
529,嘉義市,西區,蘭州二街
530,嘉義市,西區,蘭州三街
531,嘉義市,西區,蘭州四街


In [3]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def get_districts(page, city_name):
    await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")
    await page.select_option("#city", city_name)
    await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
    options = await page.locator('#cityarea option').all()
    return [await opt.get_attribute('value') for opt in options if await opt.get_attribute('value')]

async def get_roads_for_district(p, city_name, district):
    data = []
    try:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")

        # Step 1: Select city
        await page.select_option("#city", city_name)
        await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
        await asyncio.sleep(1)

        # Step 2: Select district + manually trigger the onchange event
        await page.select_option("#cityarea", district)
        await page.dispatch_event("#cityarea", "change")
        await asyncio.sleep(1)

        # Step 3: Select road name mode (Street_kind0)
        await page.check("#Street_kind0")
        await asyncio.sleep(1)

        # Step 4: Wait for roads to be loaded
        await page.wait_for_function("""
            () => {
                const el = document.querySelector('#street option');
                return el && el.value && el.value !== '';
            }
        """, timeout=10000)

        # Step 5: Scrape road options
        options = await page.locator("#street option").all()
        roads = [await opt.get_attribute("value") for opt in options if await opt.get_attribute("value")]

        for road in roads:
            data.append({
                "city": city_name,
                "district": district,
                "road": road
            })

        print(f"✅ {city_name} → {district}: {len(roads)} roads")

        await browser.close()
        return data

    except Exception as e:
        print(f"❌ {city_name} → {district}: Failed ({e})")
        return []

async def get_one_city(city_name):
    all_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # Get list of districts
        districts = await get_districts(page, city_name)
        await browser.close()

        # Scrape each district
        for district in districts:
            roads = await get_roads_for_district(p, city_name, district)
            all_data.extend(roads)

    # Save to CSV
    df = pd.DataFrame(all_data).drop_duplicates()
    df.to_csv(f"{city_name}_roads.csv", index=False)
    print(f"\n🎉 Done with {city_name}! Total roads: {len(df)}")
    return df

# Run it
await get_one_city("嘉義縣")


✅ 嘉義縣 → 番路鄉: 98 roads
✅ 嘉義縣 → 梅山鄉: 127 roads
✅ 嘉義縣 → 竹崎鄉: 186 roads
✅ 嘉義縣 → 阿里山鄉: 25 roads
✅ 嘉義縣 → 中埔鄉: 126 roads
✅ 嘉義縣 → 大埔鄉: 48 roads
✅ 嘉義縣 → 水上鄉: 92 roads
✅ 嘉義縣 → 鹿草鄉: 40 roads
✅ 嘉義縣 → 太保市: 102 roads
✅ 嘉義縣 → 朴子市: 114 roads
✅ 嘉義縣 → 東石鄉: 51 roads
✅ 嘉義縣 → 六腳鄉: 27 roads
✅ 嘉義縣 → 新港鄉: 59 roads
✅ 嘉義縣 → 民雄鄉: 165 roads
✅ 嘉義縣 → 大林鎮: 105 roads
✅ 嘉義縣 → 溪口鄉: 42 roads
✅ 嘉義縣 → 義竹鄉: 33 roads
✅ 嘉義縣 → 布袋鎮: 87 roads

🎉 Done with 嘉義縣! Total roads: 1527


,city,district,road
0,嘉義縣,番路鄉,三橋仔
1,嘉義縣,番路鄉,下石頭埔
2,嘉義縣,番路鄉,下宅仔
3,嘉義縣,番路鄉,下坑
4,嘉義縣,番路鄉,下坪仔
...,...,...,...
1522,嘉義縣,布袋鎮,樹林頭
1523,嘉義縣,布袋鎮,興中街
1524,嘉義縣,布袋鎮,龍江路
1525,嘉義縣,布袋鎮,環河街


In [4]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def get_districts(page, city_name):
    await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")
    await page.select_option("#city", city_name)
    await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
    options = await page.locator('#cityarea option').all()
    return [await opt.get_attribute('value') for opt in options if await opt.get_attribute('value')]

async def get_roads_for_district(p, city_name, district):
    data = []
    try:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")

        # Step 1: Select city
        await page.select_option("#city", city_name)
        await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
        await asyncio.sleep(1)

        # Step 2: Select district + manually trigger the onchange event
        await page.select_option("#cityarea", district)
        await page.dispatch_event("#cityarea", "change")
        await asyncio.sleep(1)

        # Step 3: Select road name mode (Street_kind0)
        await page.check("#Street_kind0")
        await asyncio.sleep(1)

        # Step 4: Wait for roads to be loaded
        await page.wait_for_function("""
            () => {
                const el = document.querySelector('#street option');
                return el && el.value && el.value !== '';
            }
        """, timeout=10000)

        # Step 5: Scrape road options
        options = await page.locator("#street option").all()
        roads = [await opt.get_attribute("value") for opt in options if await opt.get_attribute("value")]

        for road in roads:
            data.append({
                "city": city_name,
                "district": district,
                "road": road
            })

        print(f"✅ {city_name} → {district}: {len(roads)} roads")

        await browser.close()
        return data

    except Exception as e:
        print(f"❌ {city_name} → {district}: Failed ({e})")
        return []

async def get_one_city(city_name):
    all_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # Get list of districts
        districts = await get_districts(page, city_name)
        await browser.close()

        # Scrape each district
        for district in districts:
            roads = await get_roads_for_district(p, city_name, district)
            all_data.extend(roads)

    # Save to CSV
    df = pd.DataFrame(all_data).drop_duplicates()
    df.to_csv(f"{city_name}_roads.csv", index=False)
    print(f"\n🎉 Done with {city_name}! Total roads: {len(df)}")
    return df

# Run it
await get_one_city("臺南市")


✅ 臺南市 → 中西區: 129 roads
✅ 臺南市 → 東區: 259 roads
✅ 臺南市 → 南區: 140 roads
✅ 臺南市 → 北區: 85 roads
✅ 臺南市 → 安平區: 105 roads
✅ 臺南市 → 安南區: 368 roads
✅ 臺南市 → 永康區: 228 roads
✅ 臺南市 → 歸仁區: 278 roads
✅ 臺南市 → 新化區: 51 roads
✅ 臺南市 → 左鎮區: 10 roads
✅ 臺南市 → 玉井區: 47 roads
✅ 臺南市 → 楠西區: 27 roads
✅ 臺南市 → 南化區: 9 roads
✅ 臺南市 → 仁德區: 208 roads
✅ 臺南市 → 關廟區: 106 roads
✅ 臺南市 → 龍崎區: 44 roads
✅ 臺南市 → 官田區: 79 roads
✅ 臺南市 → 麻豆區: 101 roads
✅ 臺南市 → 佳里區: 120 roads
✅ 臺南市 → 西港區: 37 roads
✅ 臺南市 → 七股區: 27 roads
✅ 臺南市 → 將軍區: 24 roads
✅ 臺南市 → 學甲區: 81 roads
✅ 臺南市 → 北門區: 16 roads
✅ 臺南市 → 新營區: 191 roads
✅ 臺南市 → 後壁區: 23 roads
✅ 臺南市 → 白河區: 119 roads
✅ 臺南市 → 東山區: 72 roads
✅ 臺南市 → 六甲區: 50 roads
✅ 臺南市 → 下營區: 77 roads
✅ 臺南市 → 柳營區: 90 roads
✅ 臺南市 → 鹽水區: 68 roads
✅ 臺南市 → 善化區: 124 roads
✅ 臺南市 → 大內區: 10 roads
✅ 臺南市 → 山上區: 16 roads
✅ 臺南市 → 新市區: 104 roads
✅ 臺南市 → 安定區: 31 roads

🎉 Done with 臺南市! Total roads: 3554


,city,district,road
0,臺南市,中西區,大仁街
1,臺南市,中西區,大同路一段
2,臺南市,中西區,大勇街
3,臺南市,中西區,大埔街
4,臺南市,中西區,大涼路
...,...,...,...
3549,臺南市,安定區,嘉榮路
3550,臺南市,安定區,管寮
3551,臺南市,安定區,領寄
3552,臺南市,安定區,環福街


In [5]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def get_districts(page, city_name):
    await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")
    await page.select_option("#city", city_name)
    await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
    options = await page.locator('#cityarea option').all()
    return [await opt.get_attribute('value') for opt in options if await opt.get_attribute('value')]

async def get_roads_for_district(p, city_name, district):
    data = []
    try:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")

        # Step 1: Select city
        await page.select_option("#city", city_name)
        await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
        await asyncio.sleep(1)

        # Step 2: Select district + manually trigger the onchange event
        await page.select_option("#cityarea", district)
        await page.dispatch_event("#cityarea", "change")
        await asyncio.sleep(1)

        # Step 3: Select road name mode (Street_kind0)
        await page.check("#Street_kind0")
        await asyncio.sleep(1)

        # Step 4: Wait for roads to be loaded
        await page.wait_for_function("""
            () => {
                const el = document.querySelector('#street option');
                return el && el.value && el.value !== '';
            }
        """, timeout=10000)

        # Step 5: Scrape road options
        options = await page.locator("#street option").all()
        roads = [await opt.get_attribute("value") for opt in options if await opt.get_attribute("value")]

        for road in roads:
            data.append({
                "city": city_name,
                "district": district,
                "road": road
            })

        print(f"✅ {city_name} → {district}: {len(roads)} roads")

        await browser.close()
        return data

    except Exception as e:
        print(f"❌ {city_name} → {district}: Failed ({e})")
        return []

async def get_one_city(city_name):
    all_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # Get list of districts
        districts = await get_districts(page, city_name)
        await browser.close()

        # Scrape each district
        for district in districts:
            roads = await get_roads_for_district(p, city_name, district)
            all_data.extend(roads)

    # Save to CSV
    df = pd.DataFrame(all_data).drop_duplicates()
    df.to_csv(f"{city_name}_roads.csv", index=False)
    print(f"\n🎉 Done with {city_name}! Total roads: {len(df)}")
    return df

# Run it
await get_one_city("高雄市")


✅ 高雄市 → 新興區: 87 roads
✅ 高雄市 → 前金區: 53 roads
✅ 高雄市 → 苓雅區: 176 roads
✅ 高雄市 → 鹽埕區: 39 roads
✅ 高雄市 → 鼓山區: 137 roads
✅ 高雄市 → 旗津區: 45 roads
✅ 高雄市 → 前鎮區: 517 roads
✅ 高雄市 → 三民區: 338 roads
✅ 高雄市 → 楠梓區: 302 roads
✅ 高雄市 → 小港區: 397 roads
✅ 高雄市 → 左營區: 221 roads
✅ 高雄市 → 仁武區: 375 roads
✅ 高雄市 → 大社區: 97 roads
❌ 高雄市 → 東沙群島: Failed (Page.wait_for_function: Timeout 10000ms exceeded.)
❌ 高雄市 → 南沙群島: Failed (Page.wait_for_function: Timeout 10000ms exceeded.)
✅ 高雄市 → 岡山區: 285 roads
✅ 高雄市 → 路竹區: 92 roads
✅ 高雄市 → 阿蓮區: 61 roads
✅ 高雄市 → 田寮區: 65 roads
✅ 高雄市 → 燕巢區: 122 roads
✅ 高雄市 → 橋頭區: 385 roads
✅ 高雄市 → 梓官區: 91 roads
✅ 高雄市 → 彌陀區: 133 roads
✅ 高雄市 → 永安區: 40 roads
✅ 高雄市 → 湖內區: 45 roads
✅ 高雄市 → 鳳山區: 542 roads
✅ 高雄市 → 大寮區: 361 roads
✅ 高雄市 → 林園區: 122 roads
✅ 高雄市 → 鳥松區: 202 roads
✅ 高雄市 → 大樹區: 82 roads
✅ 高雄市 → 旗山區: 202 roads
✅ 高雄市 → 美濃區: 93 roads
✅ 高雄市 → 六龜區: 51 roads
✅ 高雄市 → 內門區: 84 roads
✅ 高雄市 → 杉林區: 36 roads
✅ 高雄市 → 甲仙區: 39 roads
✅ 高雄市 → 桃源區: 20 roads
✅ 高雄市 → 那瑪夏區: 7 roads
✅ 高雄市 → 茂林區: 3 roads
✅ 高雄市 → 茄萣區: 73 roads

🎉

,city,district,road
0,高雄市,新興區,七賢一路
1,高雄市,新興區,七賢二路
2,高雄市,新興區,八德一路
3,高雄市,新興區,八德二路
4,高雄市,新興區,大同一路
...,...,...,...
6015,高雄市,茄萣區,興達路一段
6016,高雄市,茄萣區,濱海路一段
6017,高雄市,茄萣區,濱海路二段
6018,高雄市,茄萣區,濱海路三段


In [6]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def get_districts(page, city_name):
    await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")
    await page.select_option("#city", city_name)
    await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
    options = await page.locator('#cityarea option').all()
    return [await opt.get_attribute('value') for opt in options if await opt.get_attribute('value')]

async def get_roads_for_district(p, city_name, district):
    data = []
    try:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")

        # Step 1: Select city
        await page.select_option("#city", city_name)
        await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
        await asyncio.sleep(1)

        # Step 2: Select district + manually trigger the onchange event
        await page.select_option("#cityarea", district)
        await page.dispatch_event("#cityarea", "change")
        await asyncio.sleep(1)

        # Step 3: Select road name mode (Street_kind0)
        await page.check("#Street_kind0")
        await asyncio.sleep(1)

        # Step 4: Wait for roads to be loaded
        await page.wait_for_function("""
            () => {
                const el = document.querySelector('#street option');
                return el && el.value && el.value !== '';
            }
        """, timeout=10000)

        # Step 5: Scrape road options
        options = await page.locator("#street option").all()
        roads = [await opt.get_attribute("value") for opt in options if await opt.get_attribute("value")]

        for road in roads:
            data.append({
                "city": city_name,
                "district": district,
                "road": road
            })

        print(f"✅ {city_name} → {district}: {len(roads)} roads")

        await browser.close()
        return data

    except Exception as e:
        print(f"❌ {city_name} → {district}: Failed ({e})")
        return []

async def get_one_city(city_name):
    all_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # Get list of districts
        districts = await get_districts(page, city_name)
        await browser.close()

        # Scrape each district
        for district in districts:
            roads = await get_roads_for_district(p, city_name, district)
            all_data.extend(roads)

    # Save to CSV
    df = pd.DataFrame(all_data).drop_duplicates()
    df.to_csv(f"{city_name}_roads.csv", index=False)
    print(f"\n🎉 Done with {city_name}! Total roads: {len(df)}")
    return df

# Run it
await get_one_city("屏東縣")


✅ 屏東縣 → 屏東市: 522 roads
✅ 屏東縣 → 三地門鄉: 24 roads
✅ 屏東縣 → 霧臺鄉: 21 roads
✅ 屏東縣 → 瑪家鄉: 13 roads
✅ 屏東縣 → 九如鄉: 80 roads
✅ 屏東縣 → 里港鄉: 63 roads
✅ 屏東縣 → 高樹鄉: 174 roads
✅ 屏東縣 → 鹽埔鄉: 90 roads
✅ 屏東縣 → 長治鄉: 125 roads
✅ 屏東縣 → 麟洛鄉: 65 roads
✅ 屏東縣 → 竹田鄉: 96 roads
✅ 屏東縣 → 內埔鄉: 301 roads
✅ 屏東縣 → 萬丹鄉: 176 roads
✅ 屏東縣 → 潮州鎮: 294 roads
✅ 屏東縣 → 泰武鄉: 19 roads
✅ 屏東縣 → 來義鄉: 18 roads
✅ 屏東縣 → 萬巒鄉: 62 roads
✅ 屏東縣 → 崁頂鄉: 51 roads
✅ 屏東縣 → 新埤鄉: 49 roads
✅ 屏東縣 → 南州鄉: 41 roads
✅ 屏東縣 → 林邊鄉: 37 roads
✅ 屏東縣 → 東港鎮: 148 roads
✅ 屏東縣 → 琉球鄉: 23 roads
✅ 屏東縣 → 佳冬鄉: 89 roads
✅ 屏東縣 → 新園鄉: 91 roads
✅ 屏東縣 → 枋寮鄉: 112 roads
✅ 屏東縣 → 枋山鄉: 21 roads
✅ 屏東縣 → 春日鄉: 8 roads
✅ 屏東縣 → 獅子鄉: 24 roads
✅ 屏東縣 → 車城鄉: 40 roads
✅ 屏東縣 → 牡丹鄉: 12 roads
✅ 屏東縣 → 恆春鎮: 131 roads
✅ 屏東縣 → 滿州鄉: 48 roads

🎉 Done with 屏東縣! Total roads: 3068


,city,district,road
0,屏東縣,屏東市,九江街
1,屏東縣,屏東市,三山
2,屏東縣,屏東市,三塊厝路
3,屏東縣,屏東市,上林街
4,屏東縣,屏東市,上海路
...,...,...,...
3063,屏東縣,滿州鄉,橋頭路
3064,屏東縣,滿州鄉,興海路
3065,屏東縣,滿州鄉,舊公路
3066,屏東縣,滿州鄉,欖仁路


In [7]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def get_districts(page, city_name):
    await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")
    await page.select_option("#city", city_name)
    await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
    options = await page.locator('#cityarea option').all()
    return [await opt.get_attribute('value') for opt in options if await opt.get_attribute('value')]

async def get_roads_for_district(p, city_name, district):
    data = []
    try:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")

        # Step 1: Select city
        await page.select_option("#city", city_name)
        await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
        await asyncio.sleep(1)

        # Step 2: Select district + manually trigger the onchange event
        await page.select_option("#cityarea", district)
        await page.dispatch_event("#cityarea", "change")
        await asyncio.sleep(1)

        # Step 3: Select road name mode (Street_kind0)
        await page.check("#Street_kind0")
        await asyncio.sleep(1)

        # Step 4: Wait for roads to be loaded
        await page.wait_for_function("""
            () => {
                const el = document.querySelector('#street option');
                return el && el.value && el.value !== '';
            }
        """, timeout=10000)

        # Step 5: Scrape road options
        options = await page.locator("#street option").all()
        roads = [await opt.get_attribute("value") for opt in options if await opt.get_attribute("value")]

        for road in roads:
            data.append({
                "city": city_name,
                "district": district,
                "road": road
            })

        print(f"✅ {city_name} → {district}: {len(roads)} roads")

        await browser.close()
        return data

    except Exception as e:
        print(f"❌ {city_name} → {district}: Failed ({e})")
        return []

async def get_one_city(city_name):
    all_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # Get list of districts
        districts = await get_districts(page, city_name)
        await browser.close()

        # Scrape each district
        for district in districts:
            roads = await get_roads_for_district(p, city_name, district)
            all_data.extend(roads)

    # Save to CSV
    df = pd.DataFrame(all_data).drop_duplicates()
    df.to_csv(f"{city_name}_roads.csv", index=False)
    print(f"\n🎉 Done with {city_name}! Total roads: {len(df)}")
    return df

# Run it
await get_one_city("臺東縣")


✅ 臺東縣 → 臺東市: 227 roads
✅ 臺東縣 → 綠島鄉: 16 roads
✅ 臺東縣 → 蘭嶼鄉: 6 roads
✅ 臺東縣 → 延平鄉: 12 roads
✅ 臺東縣 → 卑南鄉: 60 roads
✅ 臺東縣 → 鹿野鄉: 68 roads
✅ 臺東縣 → 關山鎮: 66 roads
✅ 臺東縣 → 海端鄉: 24 roads
✅ 臺東縣 → 池上鄉: 77 roads
✅ 臺東縣 → 東河鄉: 34 roads
✅ 臺東縣 → 成功鎮: 58 roads
✅ 臺東縣 → 長濱鄉: 36 roads
✅ 臺東縣 → 太麻里鄉: 54 roads
✅ 臺東縣 → 金峰鄉: 5 roads
✅ 臺東縣 → 大武鄉: 47 roads
✅ 臺東縣 → 達仁鄉: 24 roads

🎉 Done with 臺東縣! Total roads: 814


,city,district,road
0,臺東縣,臺東市,上海街
1,臺東縣,臺東市,大仁路
2,臺東縣,臺東市,大正路
3,臺東縣,臺東市,大同路
4,臺東縣,臺東市,大同路地下商場
...,...,...,...
809,臺東縣,達仁鄉,新生
810,臺東縣,達仁鄉,新生路
811,臺東縣,達仁鄉,新興
812,臺東縣,達仁鄉,壽𡶛


In [8]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def get_districts(page, city_name):
    await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")
    await page.select_option("#city", city_name)
    await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
    options = await page.locator('#cityarea option').all()
    return [await opt.get_attribute('value') for opt in options if await opt.get_attribute('value')]

async def get_roads_for_district(p, city_name, district):
    data = []
    try:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")

        # Step 1: Select city
        await page.select_option("#city", city_name)
        await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
        await asyncio.sleep(1)

        # Step 2: Select district + manually trigger the onchange event
        await page.select_option("#cityarea", district)
        await page.dispatch_event("#cityarea", "change")
        await asyncio.sleep(1)

        # Step 3: Select road name mode (Street_kind0)
        await page.check("#Street_kind0")
        await asyncio.sleep(1)

        # Step 4: Wait for roads to be loaded
        await page.wait_for_function("""
            () => {
                const el = document.querySelector('#street option');
                return el && el.value && el.value !== '';
            }
        """, timeout=10000)

        # Step 5: Scrape road options
        options = await page.locator("#street option").all()
        roads = [await opt.get_attribute("value") for opt in options if await opt.get_attribute("value")]

        for road in roads:
            data.append({
                "city": city_name,
                "district": district,
                "road": road
            })

        print(f"✅ {city_name} → {district}: {len(roads)} roads")

        await browser.close()
        return data

    except Exception as e:
        print(f"❌ {city_name} → {district}: Failed ({e})")
        return []

async def get_one_city(city_name):
    all_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # Get list of districts
        districts = await get_districts(page, city_name)
        await browser.close()

        # Scrape each district
        for district in districts:
            roads = await get_roads_for_district(p, city_name, district)
            all_data.extend(roads)

    # Save to CSV
    df = pd.DataFrame(all_data).drop_duplicates()
    df.to_csv(f"{city_name}_roads.csv", index=False)
    print(f"\n🎉 Done with {city_name}! Total roads: {len(df)}")
    return df

# Run it
await get_one_city("花蓮縣")

✅ 花蓮縣 → 花蓮市: 325 roads
✅ 花蓮縣 → 新城鄉: 106 roads
✅ 花蓮縣 → 秀林鄉: 45 roads
✅ 花蓮縣 → 吉安鄉: 385 roads
✅ 花蓮縣 → 壽豐鄉: 156 roads
✅ 花蓮縣 → 鳳林鎮: 121 roads
✅ 花蓮縣 → 光復鄉: 53 roads
✅ 花蓮縣 → 豐濱鄉: 37 roads
✅ 花蓮縣 → 瑞穗鄉: 148 roads
✅ 花蓮縣 → 萬榮鄉: 7 roads
✅ 花蓮縣 → 玉里鎮: 129 roads
✅ 花蓮縣 → 卓溪鄉: 16 roads
✅ 花蓮縣 → 富里鄉: 57 roads

🎉 Done with 花蓮縣! Total roads: 1585


,city,district,road
0,花蓮縣,花蓮市,一心街
1,花蓮縣,花蓮市,十六股大道
2,花蓮縣,花蓮市,三民街
3,花蓮縣,花蓮市,上海街
4,花蓮縣,花蓮市,大同街
...,...,...,...
1580,花蓮縣,富里鄉,學富路
1581,花蓮縣,富里鄉,豐南
1582,花蓮縣,富里鄉,豐南村復興
1583,花蓮縣,富里鄉,鎮寧


In [9]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def get_districts(page, city_name):
    await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")
    await page.select_option("#city", city_name)
    await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
    options = await page.locator('#cityarea option').all()
    return [await opt.get_attribute('value') for opt in options if await opt.get_attribute('value')]

async def get_roads_for_district(p, city_name, district):
    data = []
    try:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")

        # Step 1: Select city
        await page.select_option("#city", city_name)
        await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
        await asyncio.sleep(1)

        # Step 2: Select district + manually trigger the onchange event
        await page.select_option("#cityarea", district)
        await page.dispatch_event("#cityarea", "change")
        await asyncio.sleep(1)

        # Step 3: Select road name mode (Street_kind0)
        await page.check("#Street_kind0")
        await asyncio.sleep(1)

        # Step 4: Wait for roads to be loaded
        await page.wait_for_function("""
            () => {
                const el = document.querySelector('#street option');
                return el && el.value && el.value !== '';
            }
        """, timeout=10000)

        # Step 5: Scrape road options
        options = await page.locator("#street option").all()
        roads = [await opt.get_attribute("value") for opt in options if await opt.get_attribute("value")]

        for road in roads:
            data.append({
                "city": city_name,
                "district": district,
                "road": road
            })

        print(f"✅ {city_name} → {district}: {len(roads)} roads")

        await browser.close()
        return data

    except Exception as e:
        print(f"❌ {city_name} → {district}: Failed ({e})")
        return []

async def get_one_city(city_name):
    all_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # Get list of districts
        districts = await get_districts(page, city_name)
        await browser.close()

        # Scrape each district
        for district in districts:
            roads = await get_roads_for_district(p, city_name, district)
            all_data.extend(roads)

    # Save to CSV
    df = pd.DataFrame(all_data).drop_duplicates()
    df.to_csv(f"{city_name}_roads.csv", index=False)
    print(f"\n🎉 Done with {city_name}! Total roads: {len(df)}")
    return df

# Run it
await get_one_city("宜蘭縣")


✅ 宜蘭縣 → 宜蘭市: 228 roads
✅ 宜蘭縣 → 頭城鎮: 83 roads
✅ 宜蘭縣 → 礁溪鄉: 125 roads
✅ 宜蘭縣 → 壯圍鄉: 75 roads
✅ 宜蘭縣 → 員山鄉: 136 roads
✅ 宜蘭縣 → 羅東鎮: 147 roads
✅ 宜蘭縣 → 三星鄉: 301 roads
✅ 宜蘭縣 → 大同鄉: 52 roads
✅ 宜蘭縣 → 五結鄉: 156 roads
✅ 宜蘭縣 → 冬山鄉: 318 roads
✅ 宜蘭縣 → 蘇澳鎮: 211 roads
✅ 宜蘭縣 → 南澳鄉: 25 roads
❌ 宜蘭縣 → 釣魚臺: Failed (Page.wait_for_function: Timeout 10000ms exceeded.)

🎉 Done with 宜蘭縣! Total roads: 1857


,city,district,road
0,宜蘭縣,宜蘭市,一結路
1,宜蘭縣,宜蘭市,七張路
2,宜蘭縣,宜蘭市,力行路
3,宜蘭縣,宜蘭市,力新路
4,宜蘭縣,宜蘭市,三清路
...,...,...,...
1852,宜蘭縣,南澳鄉,橫山路
1853,宜蘭縣,南澳鄉,蘇花路一段
1854,宜蘭縣,南澳鄉,蘇花路二段
1855,宜蘭縣,南澳鄉,蘇花路三段


In [10]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def get_districts(page, city_name):
    await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")
    await page.select_option("#city", city_name)
    await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
    options = await page.locator('#cityarea option').all()
    return [await opt.get_attribute('value') for opt in options if await opt.get_attribute('value')]

async def get_roads_for_district(p, city_name, district):
    data = []
    try:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")

        # Step 1: Select city
        await page.select_option("#city", city_name)
        await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
        await asyncio.sleep(1)

        # Step 2: Select district + manually trigger the onchange event
        await page.select_option("#cityarea", district)
        await page.dispatch_event("#cityarea", "change")
        await asyncio.sleep(1)

        # Step 3: Select road name mode (Street_kind0)
        await page.check("#Street_kind0")
        await asyncio.sleep(1)

        # Step 4: Wait for roads to be loaded
        await page.wait_for_function("""
            () => {
                const el = document.querySelector('#street option');
                return el && el.value && el.value !== '';
            }
        """, timeout=10000)

        # Step 5: Scrape road options
        options = await page.locator("#street option").all()
        roads = [await opt.get_attribute("value") for opt in options if await opt.get_attribute("value")]

        for road in roads:
            data.append({
                "city": city_name,
                "district": district,
                "road": road
            })

        print(f"✅ {city_name} → {district}: {len(roads)} roads")

        await browser.close()
        return data

    except Exception as e:
        print(f"❌ {city_name} → {district}: Failed ({e})")
        return []

async def get_one_city(city_name):
    all_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # Get list of districts
        districts = await get_districts(page, city_name)
        await browser.close()

        # Scrape each district
        for district in districts:
            roads = await get_roads_for_district(p, city_name, district)
            all_data.extend(roads)

    # Save to CSV
    df = pd.DataFrame(all_data).drop_duplicates()
    df.to_csv(f"{city_name}_roads.csv", index=False)
    print(f"\n🎉 Done with {city_name}! Total roads: {len(df)}")
    return df

# Run it
await get_one_city("澎湖縣")


✅ 澎湖縣 → 馬公市: 154 roads
✅ 澎湖縣 → 西嶼鄉: 13 roads
✅ 澎湖縣 → 望安鄉: 9 roads
✅ 澎湖縣 → 七美鄉: 24 roads
✅ 澎湖縣 → 白沙鄉: 20 roads
✅ 澎湖縣 → 湖西鄉: 24 roads

🎉 Done with 澎湖縣! Total roads: 244


,city,district,road
0,澎湖縣,馬公市,三民路
1,澎湖縣,馬公市,三多路
2,澎湖縣,馬公市,大仁街
3,澎湖縣,馬公市,大勇街
4,澎湖縣,馬公市,大城北
...,...,...,...
239,澎湖縣,湖西鄉,湖西
240,澎湖縣,湖西鄉,湖東
241,澎湖縣,湖西鄉,隘門
242,澎湖縣,湖西鄉,鼎灣


In [11]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def get_districts(page, city_name):
    await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")
    await page.select_option("#city", city_name)
    await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
    options = await page.locator('#cityarea option').all()
    return [await opt.get_attribute('value') for opt in options if await opt.get_attribute('value')]

async def get_roads_for_district(p, city_name, district):
    data = []
    try:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")

        # Step 1: Select city
        await page.select_option("#city", city_name)
        await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
        await asyncio.sleep(1)

        # Step 2: Select district + manually trigger the onchange event
        await page.select_option("#cityarea", district)
        await page.dispatch_event("#cityarea", "change")
        await asyncio.sleep(1)

        # Step 3: Select road name mode (Street_kind0)
        await page.check("#Street_kind0")
        await asyncio.sleep(1)

        # Step 4: Wait for roads to be loaded
        await page.wait_for_function("""
            () => {
                const el = document.querySelector('#street option');
                return el && el.value && el.value !== '';
            }
        """, timeout=10000)

        # Step 5: Scrape road options
        options = await page.locator("#street option").all()
        roads = [await opt.get_attribute("value") for opt in options if await opt.get_attribute("value")]

        for road in roads:
            data.append({
                "city": city_name,
                "district": district,
                "road": road
            })

        print(f"✅ {city_name} → {district}: {len(roads)} roads")

        await browser.close()
        return data

    except Exception as e:
        print(f"❌ {city_name} → {district}: Failed ({e})")
        return []

async def get_one_city(city_name):
    all_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # Get list of districts
        districts = await get_districts(page, city_name)
        await browser.close()

        # Scrape each district
        for district in districts:
            roads = await get_roads_for_district(p, city_name, district)
            all_data.extend(roads)

    # Save to CSV
    df = pd.DataFrame(all_data).drop_duplicates()
    df.to_csv(f"{city_name}_roads.csv", index=False)
    print(f"\n🎉 Done with {city_name}! Total roads: {len(df)}")
    return df

# Run it
await get_one_city("金門縣")


✅ 金門縣 → 金沙鎮: 94 roads
✅ 金門縣 → 金湖鎮: 120 roads
✅ 金門縣 → 金寧鄉: 68 roads
✅ 金門縣 → 金城鎮: 72 roads
✅ 金門縣 → 烈嶼鄉: 39 roads
✅ 金門縣 → 烏坵鄉: 2 roads

🎉 Done with 金門縣! Total roads: 395


,city,district,road
0,金門縣,金沙鎮,三民路
1,金門縣,金沙鎮,下塘頭
2,金門縣,金沙鎮,大地
3,金門縣,金沙鎮,大堡
4,金門縣,金沙鎮,山后
...,...,...,...
390,金門縣,烈嶼鄉,楊厝
391,金門縣,烈嶼鄉,双口
392,金門縣,烈嶼鄉,羅厝
393,金門縣,烏坵鄉,大坵村


In [12]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def get_districts(page, city_name):
    await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")
    await page.select_option("#city", city_name)
    await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
    options = await page.locator('#cityarea option').all()
    return [await opt.get_attribute('value') for opt in options if await opt.get_attribute('value')]

async def get_roads_for_district(p, city_name, district):
    data = []
    try:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://www.post.gov.tw/post/internet/Postal/index.jsp?ID=207")

        # Step 1: Select city
        await page.select_option("#city", city_name)
        await page.wait_for_function("document.querySelectorAll('#cityarea option').length > 1")
        await asyncio.sleep(1)

        # Step 2: Select district + manually trigger the onchange event
        await page.select_option("#cityarea", district)
        await page.dispatch_event("#cityarea", "change")
        await asyncio.sleep(1)

        # Step 3: Select road name mode (Street_kind0)
        await page.check("#Street_kind0")
        await asyncio.sleep(1)

        # Step 4: Wait for roads to be loaded
        await page.wait_for_function("""
            () => {
                const el = document.querySelector('#street option');
                return el && el.value && el.value !== '';
            }
        """, timeout=10000)

        # Step 5: Scrape road options
        options = await page.locator("#street option").all()
        roads = [await opt.get_attribute("value") for opt in options if await opt.get_attribute("value")]

        for road in roads:
            data.append({
                "city": city_name,
                "district": district,
                "road": road
            })

        print(f"✅ {city_name} → {district}: {len(roads)} roads")

        await browser.close()
        return data

    except Exception as e:
        print(f"❌ {city_name} → {district}: Failed ({e})")
        return []

async def get_one_city(city_name):
    all_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # Get list of districts
        districts = await get_districts(page, city_name)
        await browser.close()

        # Scrape each district
        for district in districts:
            roads = await get_roads_for_district(p, city_name, district)
            all_data.extend(roads)

    # Save to CSV
    df = pd.DataFrame(all_data).drop_duplicates()
    df.to_csv(f"{city_name}_roads.csv", index=False)
    print(f"\n🎉 Done with {city_name}! Total roads: {len(df)}")
    return df

# Run it
await get_one_city("連江縣")


✅ 連江縣 → 南竿鄉: 9 roads
✅ 連江縣 → 北竿鄉: 7 roads
✅ 連江縣 → 莒光鄉: 5 roads
✅ 連江縣 → 東引鄉: 2 roads

🎉 Done with 連江縣! Total roads: 23


,city,district,road
0,連江縣,南竿鄉,仁愛村
1,連江縣,南竿鄉,介壽村
2,連江縣,南竿鄉,四維村
3,連江縣,南竿鄉,津沙村
4,連江縣,南竿鄉,珠螺村
5,連江縣,南竿鄉,馬祖村
6,連江縣,南竿鄉,清水村
7,連江縣,南竿鄉,復興村
8,連江縣,南竿鄉,福沃村
9,連江縣,北竿鄉,午沙村
